In [ ]:
import pandas as pd
import numpy as np


In [ ]:
#Read in data
f1 = ###Path to file containing vitals data for test set patients
f2 = ###Path to file containing ventilator test set patients
f3 = ###Path to file containing non-ventilator test set patients
f4=###Path to file containing COVID test set patients

vitals=pd.read_csv(f1)
VentTest=pd.read_csv(f2,header=None)
VentTest=np.array(VentTest[0].astype(int))
NoVentTest=pd.read_csv(f3,header=None)
NoVentTest=np.array(NoVentTest[0].astype(int))


Test the Models

In [ ]:
df = pd.DataFrame (columns = ['ID','Fit MSE','Fit r','Acc','Sens','Spec','PPV','FP','TP','FN','TN','N'])

In [ ]:
#run through all  remaining patients
from tensorflow import keras
#CHANGE MODEL HERE, TEST SET AS WELL TO VENT, NON VENT, or COVID
lstm=keras.models.load_model('FullEICU_trained_model_5min.h5')
for r in range(0,len(VentTest)):
    ID = VentTest[r]
    v=vitals[vitals['patientunitstayid']==ID]
    if len(v)>60:
        if ~(np.sum(v.isna()['sao2'])==len(v)):
            pt_series,vital_times=get_time_series_smooth(ID,vitals)
            #CHANGE DATA FUNCTION HERE FOR 5 VS. 30
            xtest,ytest,ts,t,n = time_data5(pt_series,vital_times)
            #only for sao2 only model
            xtest=np.array(xtest).reshape(np.shape(xtest)[0],2,1)
            pred=lstm.predict(xtest) 
            from sklearn.metrics import mean_squared_error
            ms=mean_squared_error(ytest,pred)
            from scipy import stats
            corr=stats.spearmanr(ytest,pred)
            thresh=1-np.exp((92-100)/10)
            true_hy=ytest>=thresh
            pred_hy=pred>=thresh
            from sklearn.metrics import confusion_matrix
            if np.sum(true_hy)==0:
                tp=0
                tn=len(true_hy)
                fp=np.sum(pred_hy)
                fn=0
                sens='all_neg'
                spec=tn/(tn+fp)
                acc=(tp+tn)/(tn+fp+fn+tp)
                ppv='all_neg'
            elif np.sum(true_hy)==len(true_hy):
                tp=len(true_hy)
                tn=0
                fp=0
                fn=len(pred_hy)-np.sum(pred_hy)
                sens=tp/(tp+fn)
                spec='all_pos'
                acc=(tp+tn)/(tn+fp+fn+tp)
                ppv='all_pos'
            else:    
                tn, fp, fn, tp = confusion_matrix(true_hy, pred_hy).ravel()
                spec = tn/(tn+fp)
                sens=tp/(tp+fn)
                acc=(tp+tn)/(tn+fp+fn+tp)
                if (fp==0 and tp==0):
                    ppv='no_pos_preds'
                else:
                    ppv=tp/(tp+fp)
            num=len(true_hy)
            df = df.append({'ID':ID,'Fit MSE':ms,'Fit r':corr[0],'Acc':acc,'Sens':sens,'Spec':spec,'PPV':ppv,'FP':fp,'TP':tp,'FN':fn,'TN':tn,'N':num}, ignore_index=True)
    


In [ ]:
#save the IDs
np.savetxt("TESTSET_IDs.csv", np.array(df['ID']), delimiter=",")

In [ ]:
df.to_csv('RESULTS.csv')

Waveform figures

In [ ]:
#make figures as needed
from tensorflow import keras
#replace ID, model and timedata function here!!!
ID = ###
lstm=keras.models.load_model('FullEICU_trained_model_5min.h5')
pt_series,vital_times=get_time_series_smooth(ID,vitals)
#change fxn
xtest,ytest,ts,t,n = time_data5(pt_series,vital_times)

xtest=np.array(xtest).reshape(np.shape(xtest)[0],2,1)
pred=lstm.predict(xtest) 
from sklearn.metrics import mean_squared_error
ms=mean_squared_error(ytest,pred)
from scipy import stats
corr=stats.spearmanr(ytest,pred)
import matplotlib.pyplot as plt
ytest=(np.log(-1*(ytest-1))*10) + 100
pred=(np.log(-1*(pred-1))*10) + 100
plt.rc('ytick', labelsize=16) 
plt.rc('xtick', labelsize=16) 
plt.rc('figure', titlesize=16) 
plt.rc('axes', labelsize=16)
plt.rc('legend', fontsize=16)
plt.plot(t,ytest,label='True Signal')
plt.plot(t,pred,label='Predicted Signal')
plt.xlabel('Time (min)')
plt.ylabel('SPO2')
plt.tight_layout()
plt.legend()
plt.savefig('Dataset_PtID_BestOrWorst'+str(ID)+'big.jpg',bbox_inches='tight')

Necessary Functions

In [ ]:
def get_time_series_smooth(ID,vitals):
    #pull the patients data
    vitals=vitals[vitals['patientunitstayid']==ID]
    vitals=vitals[['observationoffset','sao2']]
    vitals=vitals.fillna(method='pad')
    vitals=vitals.dropna()
    vital_times=np.array(vitals['observationoffset'])

    #smooth the necessary signals
    #from scipy.signal import savgol_filter
    sao2=vitals['sao2']
    sao2_transformed=1-np.exp((sao2-100)/10)
    sao2_smooth = np.concatenate((sao2_transformed[0:4],np.convolve(sao2_transformed, np.ones(5)/5,mode='valid')))

    
    #the patient time series
    #SaO2 only
    pt_series=np.transpose(np.vstack(sao2_smooth))

    return pt_series,vital_times

In [ ]:
def time_data5(pt_series,vital_times):
    ys=[]
    xs=[]
    t=[]
    ts=2
    inputshape=0
    #for SaO2 model only
    for i in range(0,len(pt_series[0])-(ts+1)):
        y=pt_series[0][i+ts]
        t.append(vital_times[i+ts])
        #all
        x=pt_series[0][i:i+ts]
        ys.append(y)
        xs.append(x)
    ys=np.transpose(ys)
    n=0
    return xs,ys,ts,t,n

In [ ]:
def time_data30(pt_series,vital_times):
    ys=[]
    xs=[]
    t=[]
    ts=2
    inputshape=0
    #for SaO2 model only
    for i in range(0,len(pt_series[0])-(ts+1)):
        y=pt_series[0][i+ts]
        t.append(vital_times[i+ts])
        #all
        x=pt_series[0][i:i+ts]
        ys.append(y)
        xs.append(x)
    ys=np.transpose(ys)
    n=0
    #downsampling
    d=6
    xs=xs[::d]
    ys=ys[::d]
    t=t[::d]
    return xs,ys,ts,t,n